In [ ]:
laminar = model.arr_latter.T

array = np.zeros((laminar.shape[0],laminar.shape[1]*2-1), dtype=np.float64)
array[:, 0] = np.real(laminar[:, -1])
array[:,1::2] = np.real(laminar[:,:-1])
array[:,2::2] = np.imag(laminar[:,:-1])
np.savetxt("stable_laminar_50000period_beta0.417_nu0.00017322774042408294_step0.01.txt", array)

In [ ]:
laminar = np.load("/Users/hibiki/Downloads/beta0.46711_nu0.00011871562745803017_step0.01_3000check_500pro_20000period.npz")["laminar"].T

array = np.zeros((laminar.shape[0],laminar.shape[1]*2-1), dtype=np.float64)
array[:, 0] = np.real(laminar[:, -1])
array[:,1::2] = np.real(laminar[:,:-1])
array[:,2::2] = np.imag(laminar[:,:-1])
np.savetxt("loosely_generated_laminar_20000period_beta0.47611_nu0.00011871562745803017_step0.01.txt", array)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import cmath
from numba import njit, f8,i8,c16
import numba
import matplotlib.animation as animation
from IPython.display import HTML
from IPython.display import Video
import copy
import matplotlib
import concurrent.futures
import itertools
#import banpei
import func_for_paralell
import cv2
import glob
import os
import sys
sys.path.append('../../lb')
import RungeKutta_acceralated


#パラメーター
nu = 0.00018

f = (1 + 1j) * 5 * 0.001

Dimention = 14 #uの次元数
q = 2
k_0 = 2 ** (-4)
k_n = np.zeros((Dimention+4,1))
k_n[2:-2,0] = [k_0 * q ** (i+1) for i in range(Dimention)]
#print(k_n)

c_n_1 = copy.deepcopy(k_n)[2:-2, 0:]
c_n_1[-2:, 0:] = np.zeros((2,1))
#print(c_n_1)

beta = 0.417
c_n_2 = copy.deepcopy(k_n)[1:-3, 0:] * -beta
c_n_2[-1, 0] = 0
# print(c_n_2)

c_n_3 = copy.deepcopy(k_n)[:-4, 0:] * (beta - 1)
#print(c_n_3)



c_n_3 = copy.deepcopy(k_n)[2:-2, 0:] * (beta - 1)
c_n_3[:2, 0:] = np.zeros((2,1))
#print(c_n_3)

#parameter = [nu,f,k_n,c_n_1,c_n_2,c_n_3]
parameter = {
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}
#print(parameter)
#print(k_n)

runge_para = {
    'start' :  np.array([
        [0.4350E+00 + 0.5008E+00j],
        [0.1259E+00 + 0.2437E+00j],
        [-0.8312E-01 + -0.4802E-01j],
        [0.5164E-01 + -0.1599E+00j],
        [-0.1899E+00 + -0.3602E-01j],
        [0.4093E-03 + 0.8506E-01j],
        [0.9539E-01 + 0.3215E-01j],
        [-0.5834E-01 + 0.4433E-01j],
        [-0.8790E-02 + 0.2502E-01j],
        [0.3385E-02 + 0.1148E-02j],
        [-0.7072E-04 + 0.5598E-04j],
        [-0.5238E-07 + 0.1467E-06j],
        [0.1E-07 + 0.1E-06j],
        [0.1E-07 + 0.1E-06j]
    ]),
    'step' : 0.001,
    'period' : (0,5000),
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}

def comb_movie(movie_files,out_path):

    # 形式はmp4
    fourcc = cv2.VideoWriter_fourcc('m','p','4','v')

    # 動画情報の取得
    movie = cv2.VideoCapture(movie_files[0])
    fps = movie.get(cv2.CAP_PROP_FPS)
    height = movie.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = movie.get(cv2.CAP_PROP_FRAME_WIDTH)


    # 出力先のファイルを開く
    out = cv2.VideoWriter(out_path, int(fourcc), fps, (int(width), int(height)))


    for movies in (movie_files):
        print(movies)
        # 動画ファイルの読み込み，引数はビデオファイルのパス
        movie = cv2.VideoCapture(movies)

        # 正常に動画ファイルを読み込めたか確認
        if movie.isOpened() == True: 
            # read():1コマ分のキャプチャ画像データを読み込む
            ret, frame = movie.read() 
        else:
            ret = False

        while ret:
            # 読み込んだフレームを書き込み
            out.write(frame)
            # 次のフレーム読み込み
            ret, frame = movie.read()

# 計算(時系列プロット)

In [ ]:
%%time
#縦軸に各変数、横軸時間　×次元数 + flux
# runge_para["period"]=(0,10000)
runge_para["step"] = 0.001
skip=1
# runge_para['start'] = np.load("../cpp/initials/beta0.422_nu0.00018_2000period.npy").reshape(14,1)
runge_para["nu"] = 0.0022
# #betaの値に従いc_n_2,c_n_3を更新
# beta = 0.416

#flux of Nth shell
#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))
#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
%time model.get_arr_latter(**runge_para,latter=1)

plt.figure(figsize=(20,30))
for i in range(Dimention):
    plt.subplot(Dimention+1,1,i+1)
    plt.ylabel(f"u{i+1}")
    #plt.yscale=("log")
    plt.plot(np.real(model.arr_latter[-1,::skip]),abs(model.arr_latter[i,::skip]),lw=1)

# plt.subplot(Dimention+1,1,Dimention+1)
# plt.ylabel(f"flux of {Nth_shell}th shell")
# plt.plot(np.real(model.arr_latter[-1,::skip]), abs(RungeKutta_acceralated.Long_laminar.flux(model.arr_latter[:-1,:], Nth_shell, beta)[0,::skip]))
#plt.subplot(Dimention,1,1)
# plt.ylabel(f"u{4}")
# plt.yscale=("log")
# plt.plot(np.real(model.arr_latter[-1,::skip]),abs(model.arr_latter[3,::skip]),lw=0.1)

plt.savefig(fname=f"../attracter/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_time{runge_para['period']}_{Dimention}dim_flux_of_{Nth_shell}th.png")

In [ ]:
np.load("../cpp_codes/initials/beta0.422_nu0.00018_2000period.npy").shape

## フーリエ変換

In [ ]:
#フーリエ変換
plt.figure(figsize=(50,30))
N = model.arr_latter.shape[1]
dt = runge_para['step']
F = np.fft.fft(model.arr_latter[0,:]) # 変換結果
freq = np.fft.fftfreq(N, d=dt) # 周波数

Amp = np.abs(F/(N/2)) # 振幅

fig, ax = plt.subplots()
ax.plot(freq, Amp)
ax.set_xlabel("Freqency [Hz]")
ax.set_ylabel("Amplitude")
ax.grid()
plt.show()


# 縦軸に各変数、横軸時間　×次元数(長時間分割ver)

In [ ]:
runge_para["period"]=(0,5000)
#periodの続きを何回するか(初回も含む)

mouikkai_asoberu_don = 10


#刻み幅
runge_para["step"] = 0.01

skip=100

runge_para["start"] = np.load("nu0.00017795206429227755_beta0.41676767676767673_0.0114dim.npy")

#nu
runge_para["nu"] = 0.00017795206429227755
#betaの値に従いc_n_2,c_n_3を更新
beta = 0.41676767676767673

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()

for big_i in tqdm(range(mouikkai_asoberu_don)):
    model.get_arr_latter(**runge_para,latter=1)

    plt.figure(figsize=(20,30))
    for i in range(Dimention):
        plt.subplot(Dimention,1,i+1)
        plt.xticks(np.arange(runge_para['period'][0],runge_para['period'][1],step=5000))
        plt.ylabel(f"u{i+1}")
        plt.plot(np.real(model.arr_latter[-1,::skip]),abs(model.arr_latter[i,::skip]),lw=1)
    
    plt.savefig(fname=f"../long_attracter/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_time({int(abs(model.arr_latter[-1,0])),int(abs(model.arr_latter[-1,-1]))})_{Dimention}dim.png")
        
    plt.figure(figsize=(20,30))
    for i in range(10):
        plt.subplot(10,1,i+1)
        plt.xticks(np.arange(np.abs(model.arr_latter[-1,len(model.arr_latter[-1,:])//10*i]),np.abs(model.arr_latter[-1,len(model.arr_latter[-1,:])//10*(i+1)-1]),1000))
        plt.ylabel(f"u{4}")
        plt.plot(np.real(model.arr_latter[-1,len(model.arr_latter[-1,:])//10*i:len(model.arr_latter[-1,:])//10*(i+1)-1:skip]),abs(model.arr_latter[i,len(model.arr_latter[-1,:])//10*i:len(model.arr_latter[-1,:])//10*(i+1)-1:skip]),lw=1)
    
    plt.savefig(fname=f"../long_attracter/4th_dim_enlarged_nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_time({int(abs(model.arr_latter[-1,0])),int(abs(model.arr_latter[-1,-1]))}).png")
    
    cut_arr = loc_max_7(model.arr_latter,cut_arr)
    
    #続きをやるためにパラメータ更新
    runge_para['start'] = model.arr_latter[:-1,-1:]
    runge_para['period'] = (int(abs(model.arr_latter[-1,-1])),int(abs(model.arr_latter[-1,-1]))*2-int(abs(model.arr_latter[-1,0])))


In [ ]:
#セーブ
np.savez_compressed(f"../numpy_array/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_time{0,int(abs(model.arr_latter[-1,-1]))}_{Dimention}dim.npz", array = cut_arr[:,1:])

# アニメーション時系列

In [ ]:
runge_para['period']=(0,10000)
runge_para['step'] = 0.01

#beta_nuのパラメータ候補作成
beta_begin = 0.5 #パラメータ開始
beta_end = 0.41 #パラメータ終わり

nu_begin = 0.00018
nu_end = 0.00018

param_step = 100 #パラメータ何回刻む
latter = 1 #後半何分の1をプロットするか
skip = 10 #各パラメータでプロットする際、何刻み分飛ばしでプロットするか(大きければカクカク。メモリ対策)
#参考までにdt0.00025,9000-10000時間では1000skipぐらいなら気にならない。10000だと酷い。

beta_param = np.linspace(beta_begin,beta_end,param_step) #betaパラメータndarray作成
nu_param = np.linspace(nu_begin,nu_end,param_step) 
#nu_param=[math.pow(10,i) for i in nu_param] #log10の時は実行
step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1
#kaishi = step_number // latter

laminar = np.load("ラミナー軌道例.npy")
 

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
#プロットの設定
fig, ax = plt.subplots(figsize=(40,10))
ims = []
#ax.set_xlabel("|u4|")
#ax.set_ylabel("|u5|")

for i in tqdm(range(param_step)):
    #nu更新
    runge_para["nu"] = nu_param[i]
    
    #betaの値に従いc_n_2,c_n_3を更新
    #c_n_2
    runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta_param[i]
    runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))
    
    #c_n_3
    runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta_param[i] - 1)
    
    
    model.get_arr_latter(**runge_para,latter=latter)

    # #次のパラメータの初期値を今回の最後の値に変更
    # runge_para["start"] = model.arr[:-1,-1:]
    
    text = ax.text(0.5,1.05,(f"beta={beta_param[i]} \n nu={nu_param[i]}"), size = 15, color = "green", ha = "center", transform=ax.transAxes)
    
    im1 = ax.plot(abs(model.arr_latter[-1,::skip]),abs(model.arr_latter[2,::skip]),color="b",linewidth=0.1)
    #im2 = ax.plot(abs(laminar[3,200000::10]),abs(laminar[4,200000::10]),color="r",linewidth=0.1,alpha = 0.1)
    ims.append(im1 +[text])

anim = animation.ArtistAnimation(fig, ims)
anim.save(f"../animation_attracter_per_parameter/nu({nu_param[0]},{nu_param[-1]})_beta({beta_param[0]},{beta_param[-1]})_{runge_para['step']}_{runge_para['period']}_{param_step}cut_{Dimention}dim_latter{round(1/latter)}.mp4")
HTML(anim.to_jshtml())

#　アニメーション(2軸プロット・パラメータ変更)

In [ ]:
#アニメーション(パラメータ変更)

runge_para['period']=(0,50000)
#runge_para['start'] = np.load("間欠カオス?初期値beta0.47511_nu0.00013018004201420142.npy")
runge_para['step'] = 0.01

#beta_nuのパラメータ候補作成
beta_begin = 0.467#パラメータ開始
beta_end = 0.47 #パラメータ終わり

nu_begin = 0.0001187156
nu_end = 0.000118

param_step = 100 #パラメータ何回刻む
latter = 10 #後半何分の1をプロットするか
skip = 10 #各パラメータでプロットする際、何刻み分飛ばしでプロットするか(大きければカクカク。メモリ対策)
#参考までにdt0.00025,9000-10000時間では1000skipぐらいなら気にならない。10000だと酷い。

beta_param = np.linspace(beta_begin,beta_end,param_step) #betaパラメータndarray作成
nu_param = np.linspace(nu_begin,nu_end,param_step) 
#nu_param=[math.pow(10,i) for i in nu_param] #log10の時は実行
step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1

laminar = np.load("ラミナー軌道例.npy")
 

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()

#プロットの設定
fig, ax = plt.subplots(dpi=100)
ims = []
ax.set_xlabel("|u4|")
ax.set_ylabel("|u5|")

for i in tqdm(range(param_step)):
    #nu更新
    runge_para["nu"] = nu_param[i]
    
    #betaの値に従いc_n_2,c_n_3を更新
    #c_n_2
    runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta_param[i]
    runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))
    
    #c_n_3
    runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta_param[i] - 1)
    
    
    model.get_arr_latter(**runge_para,latter=latter)

    # #次のパラメータの初期値を今回の最後の値に変更
    # runge_para["start"] = model.arr[:-1,-1:]
    
    text = ax.text(0.5,1.05,(f"beta={beta_param[i]} \n nu={nu_param[i]}"), size = 15, color = "green", ha = "center", transform=ax.transAxes)
    
    im1 = ax.plot(abs(model.arr_latter[3,::skip]),abs(model.arr_latter[4,::skip]),color="b",linewidth=0.1)
    im2 = ax.plot(abs(laminar[3,200000::10]),abs(laminar[4,200000::10]),color="r",linewidth=0.1,alpha = 0.1)
    ims.append(im1 + im2 +[text])

anim = animation.ArtistAnimation(fig, ims)
anim.save(f"../animation_attracter_per_parameter/nu({nu_param[0]},{nu_param[-1]})_beta({beta_param[0]},{beta_param[-1]})_{runge_para['step']}_{runge_para['period']}_{param_step}cut_{Dimention}dim_latter{round(1/latter)}.mp4")
HTML(anim.to_jshtml())

In [ ]:
#アニメーション(パラメータ変更)
shell1 = 4
shell2 = 5

runge_para['period']=(0,10000)
runge_para['start'] = np.load("間欠カオス?初期値beta0.47511_nu0.00013018004201420142.npy")
runge_para['step'] = 0.01

#beta_nuのパラメータ候補作成
beta_begin = 0.4#パラメータ開始
beta_end = 0.5 #パラメータ終わり

nu_begin = 0.0002
nu_end = 0.00012

param_step = 100 #パラメータ何回刻む
latter = 2 #後半何分の1をプロットするか
skip = 10 #各パラメータでプロットする際、何刻み分飛ばしでプロットするか(大きければカクカク。メモリ対策)
#参考までにdt0.00025,9000-10000時間では1000skipぐらいなら気にならない。10000だと酷い。

beta_param = list(np.linspace(beta_begin,beta_end,param_step)) #betaパラメータndarray作成
nu_param = list(np.linspace(nu_begin,nu_end,param_step))
#nu_param=[math.pow(10,i) for i in nu_param] #log10の時は実行

workers = 6

runge_paras = [runge_para for _ in range(workers)]
split_betas = [beta_param[i:i+math.ceil(param_step/workers)] for i in range(0, param_step, math.ceil(param_step/workers))]
split_nus = [nu_param[i:i+math.ceil(param_step/workers)] for i in range(0, param_step, math.ceil(param_step/workers))]
param_num = [len(i) for i in split_betas]


with concurrent.futures.ProcessPoolExecutor(max_workers = 8) as executor:
    rsl = executor.map(func_for_paralell.animation_per_parameter, itertools.repeat(runge_para), split_betas, split_nus, param_num, itertools.repeat(latter), itertools.repeat(skip), itertools.repeat(shell1), itertools.repeat(shell2))

results=[_ for _ in rsl]
for i in range(workers):
    anim = animation.ArtistAnimation(results[i][0], results[i][1])
    anim.save(f"../animation_attracter_per_parameter/anim{i}.mp4")
    plt.close()
    
# ディレクトリ内の動画をリストで取り出す
files = [f"../animation_attracter_per_parameter/anim{i}.mp4" for i in range(workers)]

# 出力ファイル名
out_path = f"../animation_attracter_per_parameter/nu({nu_param[0]},{nu_param[-1]})_beta({beta_param[0]},{beta_param[-1]})_{runge_para['step']}_{runge_para['period']}_{param_step}cut_{Dimention}dim_latter{round(1/latter)}.mp4"

comb_movie(files,out_path)

for i in range(workers):
    os.remove(f"../animation_attracter_per_parameter/anim{i}.mp4")
    
Video(filename = f"../animation_attracter_per_parameter/nu({nu_param[0]},{nu_param[-1]})_beta({beta_param[0]},{beta_param[-1]})_{runge_para['step']}_{runge_para['period']}_{param_step}cut_{Dimention}dim_latter{round(1/latter)}.mp4",html_attributes = 'controls')

# 運動アニメーション

In [ ]:
#軌道(運動アニメーション・二軸)
runge_para["period"]=(0,10000)
runge_para['step'] = 0.01
runge_para['start'] = np.load("14次元初期値.npy")
window = 100
matplotlib.rcParams['animation.embed_limit'] = 2**128

#nu
runge_para["nu"] = 0.00018

#betaの値に従いc_n_2,c_n_3を更新
beta= 0.417

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
model.get_arr(**runge_para)

#プロットの設定
fig, ax = plt.subplots(dpi=100)
ims = []
ax.set_xlabel("|u4|")
ax.set_ylabel("|u5|")

#アニメーション
for i in tqdm(range(len(model.arr[-1,:])//window-3)):
    # text = ax.text(0.5,1.05,(f"time={abs(model.arr[-1,window*i])} \n flux of energy through the 4th shell = {flux(model.arr[:-1, i*window:i*window+1], 1, beta)}"), size = 15, color = "green", ha = "center", transform=ax.transAxes)
    text = ax.text(0.5,1.05,f"time={abs(model.arr[-1,window*i])}", size = 15, color = "green", ha = "center", transform=ax.transAxes)
    im = ax.plot(abs(model.arr[3,i*window:(i+3)*window]),abs(model.arr[4,i*window:(i+3)*window]),color="b",linewidth=1)
    ims.append(im+[text])

anim = animation.ArtistAnimation(fig, ims)
anim.save(f"../animation_orbit/beta={beta}_nu={runge_para['nu']}_{runge_para['step']}_{runge_para['period']}_{Dimention}dim_.mp4")
HTML(anim.to_jshtml())

In [ ]:
#軌道(画像)(上のとセット)

latter = 10
step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1
kaishi = step_number // latter
skip = 10


#プロットの設定
fig, ax = plt.subplots(dpi=1000)
ax.set_xlabel("|u4|")
ax.set_ylabel("|u5|")


ax.plot(abs(model.arr_latter[3,-kaishi::skip]),abs(model.arr_latter[4,-kaishi::skip]),color="b",linewidth=0.1)

fig.savefig(fname=f"../attracter/orbit_nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_time{runge_para['period']}_{Dimention}dim_latter{round(1/latter,3)}.png")

In [ ]:
runge_para['start']

In [ ]:
#横軸時間でプロット
#軌道(運動アニメーション)
runge_para["period"]=(0,10000)
runge_para['step'] = 0.01
runge_para['start'] = np.load("14次元初期値.npy")
runge_para['start'][3] += 0.1+0.1j
window = 10000
speed = 1000
matplotlib.rcParams['animation.embed_limit'] = 2**128

#nu
runge_para["nu"] = 0.00018

#betaの値に従いc_n_2,c_n_3を更新
beta= 0.41616

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
model.get_arr(**runge_para)

# Define the update function
def update(frame):
    ax1.cla()
    ax2.cla()
    ax1.set_ylabel("|u4|")
    ax2.set_ylabel("|u5|")
    ax1.plot(abs(model.arr[-1, frame*speed:frame*speed+window]), abs(model.arr[3, frame*speed:frame*speed+window]), color="b", linewidth=0.1)
    ax2.plot(abs(model.arr[-1, frame*speed:frame*speed+window]), abs(model.arr[4, frame*speed:frame*speed+window]), color="b", linewidth=0.1)

# Create the figure and axes
fig, (ax1, ax2) = plt.subplots(2, 1)

# Set the labels for the axes
ax1.set_ylabel("|u4|")
ax2.set_ylabel("|u5|")

ax1.set_ylim(0, 0.4)
ax2.set_ylim(0,0.4)

# Create the animation
anim = animation.FuncAnimation(fig, update, frames=len(model.arr[-1,:]-window+1)//speed, interval=200)

# Save the animation as an MP4 file
anim.save(f"../animation_orbit/beta={beta}_nu={runge_para['nu']}_{runge_para['step']}_{runge_para['period']}_{Dimention}dim_.mp4")

# Display the animation as HTML
HTML(anim.to_jshtml())


In [ ]:
# Define the update function
def update(frame):
    ax1.cla()
    ax2.cla()
    ax1.set_ylabel("|u4|")
    ax2.set_ylabel("|u5|")
    ax1.plot(abs(model.arr[-1, frame*speed:frame*speed+window]), abs(model.arr[3, frame*speed:frame*speed+window]), color="b", linewidth=0.1)
    ax2.plot(abs(model.arr[-1, frame*speed:frame*speed+window]), abs(model.arr[4, frame*speed:frame*speed+window]), color="b", linewidth=0.1)

# Create the figure and axes
fig, (ax1, ax2) = plt.subplots(2, 1)

# Set the labels for the axes
ax1.set_ylabel("|u4|")
ax2.set_ylabel("|u5|")

ax1.set_ylim(0, 0.4)
ax2.set_ylim(0,0.4)

# Create the animation
anim = animation.FuncAnimation(fig, update, frames=len(model.arr[-1,:]-window+1)//speed, interval=200)

# Save the animation as an MP4 file
anim.save(f"../animation_orbit/beta={beta}_nu={runge_para['nu']}_{runge_para['step']}_{runge_para['period']}_{Dimention}dim_.mp4")

# Display the animation as HTML
HTML(anim.to_jshtml())


In [ ]:
%%time
#軌跡(画像・二軸プロット)
runge_para["period"]=(0,50000)

latter = 10
skip = 1000

#nu
runge_para["nu"] = 0.0001773323130186352

#betaの値に従いc_n_2,c_n_3を更新
beta=0.4162626262626263


#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1
#kaishi = step_number // latter

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
model.get_arr_latter(**runge_para,latter=latter)

#プロットの設定
fig, ax = plt.subplots(dpi=100)
ax.set_xlabel("|u4|")
ax.set_ylabel("|u5|")


ax.plot(abs(model.arr_latter[3,::skip]),abs(model.arr_latter[4,::skip]),color="b",linewidth=0.1)

fig.savefig(fname=f"../attracter/orbit_beta{beta}_nu{runge_para['nu']}_{runge_para['step']}_time{runge_para['period']}_{Dimention}dim_latter{round(1/latter,2)}.png")

# 時間遅れ

In [ ]:
#第N変数の時間遅れプロット
runge_para["period"]=(0,30000)
runge_para["step"] = 0.00025

latter = 4
skip = 1000
delay =  #時間遅れ(この値をstepで割ってスライスする)
N = 4 #第何変数

#nu
runge_para["nu"] = 0.0001746309482023122
#betaの値に従いc_n_2,c_n_3を更新
beta=  0.41404040404040404

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1
#kaishi = step_number // latter

# #インスタンス化
# model = RungeKutta_acceralated.RungeKutta()
# model.get_arr_latter(**runge_para,latter=latter)

#プロットの設定
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')

ax.set_xlabel("t")
ax.set_ylabel("t-$\u03c4$")
ax.set_zlabel("t-2$\u03c4$")
ax.set_title(f"U_{N},$\u03c4$={delay} \n beta = {beta} \n nu = {runge_para['nu']} \n period({abs(model.arr_latter[-1,0])},{abs(model.arr_latter[-1,-1])})")
    
ax.plot(abs(model.arr_latter[N-1,int(2*delay//runge_para['step'])::skip]),abs(model.arr_latter[N-1,int(delay//runge_para['step']):-int(delay//runge_para['step']):skip]),abs(model.arr_latter[N-1,:-int(2*delay//runge_para['step']):skip]),color="b",linewidth=0.1)

fig.savefig(fname=f"../laminar/delayed_orbit_beta{beta}_nu{runge_para['nu']}_{runge_para['step']}_time{runge_para['period']}_{delay}delay_{Dimention}dim_latter{round(1/latter,2)}.png")

In [ ]:
#第N変数の時間遅れプロット
runge_para["period"]=(0,25000)
runge_para["step"] = 0.00025

latter = 10
skip = 1000
delay_list = [i for i in range(1,100)] #時間遅れ(この値をstepで割ってスライスする)
N = 4 #第何変数

#nu
runge_para["nu"] = 0.0001746309482023122
#betaの値に従いc_n_2,c_n_3を更新
beta=  0.41404040404040404

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1
#kaishi = step_number // latter

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
model.get_arr_latter(**runge_para,latter=latter)

#プロットの設定
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')

ax.set_xlabel("t")
ax.set_ylabel("t-$\u03c4$")
ax.set_zlabel("t-2$\u03c4$")

ims = []
for delay in tqdm(delay_list):
    
    text = ax.text(0,0,0.6,f"U_{N},$\u03c4$={delay} \n beta = {beta} \n nu = {runge_para['nu']} \n period({abs(model.arr_latter[-1,0])},{abs(model.arr_latter[-1,-1])})")
    
    im = ax.plot(abs(model.arr_latter[N-1,int(2*delay//runge_para['step'])::skip]),abs(model.arr_latter[N-1,int(delay//runge_para['step']):-int(delay//runge_para['step']):skip]),abs(model.arr_latter[N-1,:-int(2*delay//runge_para['step']):skip]),color="b",linewidth=0.1)
    ims.append(im+[text])

anim = animation.ArtistAnimation(fig, ims)
anim.save(f"../laminar/delayed_orbit_beta{beta}_nu{runge_para['nu']}_{runge_para['step']}_time{int(abs(model.arr_latter[-1,0])),int(abs(model.arr_latter[-1,-1]))}_{Dimention}dim_latter{round(1/latter,2)}.mp4")
HTML(anim.to_jshtml())

In [ ]:
#u_4が0.2を下から通る時のu_nをndarrayで返す関数
@njit(f8[:](c16[:,:],i8,i8,f8[:],f8),cache=True)
def cutter(arr,step_number,dimention,z_list,cutter_num):
    for j in range(len(arr[0,:]) - 1):
        #7点とって極大判定
        if abs(arr[3,j]) < cutter_num and abs(arr[3,j+1]) > cutter_num:
            z_list = np.append(z_list,abs(arr[dimention-1,j+1]))
    return z_list

In [ ]:
%%time
runge_para["period"]=(0,25000)

latter = 10

#プロットする変数
var1 = 5
var2 = 6

#第四変数を切る数
cutting_num = 0.1

#nu
runge_para["nu"] = 0.0001746309482023122
#betaの値に従いc_n_2,c_n_3を更新
beta=  0.41404040404040404

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1

#インスタンス化
#model = RungeKutta_acceralated.RungeKutta()
#model.get_arr_latter(**runge_para,latter=latter)

#プロットの設定
plt.figure(figsize=(10,10))
plt.xlabel(f"|u{var1}|")
plt.ylabel(f"|u{var2}|")

x_list=np.array([],dtype=np.float64)#var1を入れる空のリスト
y_list=np.array([],dtype=np.float64)#var2を入れる空のリスト

x_list = cutter(model.arr_latter,step_number,var1,x_list,cutting_num)
y_list = cutter(model.arr_latter,step_number,var2,y_list,cutting_num)

plt.scatter(x_list, y_list,s=10)

plt.savefig(fname=f"../poincare/beta{beta}_nu{runge_para['nu']}_{runge_para['step']}step_time{(model.arr_latter[-1,0],model.arr_latter[-1,-1])}_{Dimention}dim_cutter{cutting_num}_U{var1}andU{var2}.png")